In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import jwt
import time
import requests

f = open("private_key.pem", "r")

private_key = f.read()

client_id = "s8q6m29leq6h6rgmd1tkhu15o0"
signing_key_id = "beA3jYM9VnCeao-c_--LPrWHKnoG4s8ST0eNBmlkQ0A"

api_url = "https://api.meetup.com/gql"


CSV_FILENAME = "leap_events_socal.csv"
MD_FILENAME = "leap_events_socal.md"

In [2]:

headers_jwt = {
    "alg": "RS256",
    "typ": "JWT",
    "kid": signing_key_id
}


payload = {
    "iss": client_id,
    "sub": client_id,
    "aud": "https://www.meetup.com/",
    "iat": int(time.time()),
    "exp": int(time.time()) + 300  
}


jwt_token = jwt.encode(payload, private_key, algorithm="RS256", headers=headers_jwt)

In [3]:
def make_event_query(event_id):
    return f"""
    query {{
      event(id: "{event_id}") {{
        title
        description
        eventUrl
        dateTime
        group {{
          city
          organizer{{
            name
          }}
        }}
        venues {{
          name
        }}
        feeSettings{{
          amount
        }}
      }}
    }}
    """

In [4]:
def run_graphql_query(query, variables=None):
    headers = {
        "Authorization": f"Bearer {jwt_token}",
        "Content-Type": "application/json"
    }

    payload = {
        "query": query,
        "variables": variables or {}
    }

    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise error if request failed

    return response.json()

In [5]:
def clean_description(html):
    """Remove HTML tags from event description."""
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text().strip()


In [6]:
def parse_event(event):
    """Transform the event data into a dictionary."""
    title = event["title"]
    orig_name = event["group"]["organizer"]["name"]
    date = event["dateTime"]
    hour = date[11:13]
    convert = int(hour)
    new = convert % 12 ## convert to military time
    time = str(new) + date[13:]
    date = date[0:10]
    city = event["group"]["city"]

    if event["venues"]:
        venue = event["venues"][0]["name"]  
    else:
        venue = "Unknown"

    description_html = event["description"]
    description = clean_description(description_html)
    url = event["eventUrl"]
    if event["feeSettings"]:
        fee = event["feeSettings"]["amount"]
    else:
        fee = 0
    
    return {
        "Title": title,
        "Organizer Name": orig_name,
        "Date": date,
        "Time": time,
        "City": city,
        "Venue": venue,
        "Description": description,
        "URL": url,
        "Fee": fee
    }

In [7]:
def write_csv(events, filename):
    """Save to CSV"""
    df = pd.DataFrame(events)
    df.to_csv(filename, index=False)
    

In [8]:
def write_markdown(events, filename):
    """Save to Markdown"""
    event_number = 1
    f = open(filename, "w", encoding="utf-8")
    
    f.write("#Career Planning and Leadership Events in Southern California\n\n")
    for event in events:
        f.write("#" + str(event_number) + ": " + event["Title"] + "\n\n")
        event_number += 1
        f.write("Organizer Name: " + event["Organizer Name"] + "\n\n")
        f.write("Date: " + event["Date"] + "\n\n")
        f.write("Time: " + event["Time"] + " PT\n\n") 
        f.write("City: " + event["City"]+ "\n\n")
        f.write("Venue: " + event["Venue"]+ "\n\n")
        desc = event["Description"]
        first_paragraph = desc.split("\n\n")[0]  # Gets the first paragraph
        f.write("Description: " + first_paragraph + "...\n\n")
        f.write("URL: " + event["URL"] + "\n\n")
        f.write("Fee: $" + str(event["Fee"]) + "\n\n")

In [9]:
query = """
query($filter: SearchConnectionFilter!, $input: ConnectionInput) {
  keywordSearch(filter: $filter, input: $input) {
    edges {
      node {
        id
      }
    }
    pageInfo {
      hasNextPage
      endCursor
    }
  }
}
"""
filter_input = {
  "filter": {
    "query": "career",
    "lat": 33.85,
    "lon": -117.95,
    "radius": 50,
    "source": ["EVENTS"]
  },
  "first": 20
}

In [10]:
if __name__ == "__main__":
    result = run_graphql_query(query, filter_input)
    IDS = []
    edges = result["data"]["keywordSearch"]["edges"]

    for edge in edges:
        node = edge["node"]
        IDS += [node["id"]]

    parsed_events = []  
    
    for i in range(len(IDS)):
        id = IDS[i]
        event_string = make_event_query(id)
        response = run_graphql_query(event_string)
        event = response["data"]["event"]
        parsed_event = parse_event(event)
        parsed_events = parsed_events + [parsed_event]
        

    

    write_csv(parsed_events, CSV_FILENAME)
    write_markdown(parsed_events, MD_FILENAME)